# Optical Properties: GW and BSE Calculations

This notebook demonstrates how to calculate optical properties using many-body perturbation theory.
You will learn how to:

1. Run GW quasiparticle calculations for accurate band gaps
2. Calculate BSE optical absorption spectra
3. Plot and analyze optical spectra
4. Compare DFT vs GW band gaps
5. Extract exciton binding energies

**Theory Background:**
- **GW approximation**: Corrects DFT eigenvalues to quasiparticle energies
- **BSE (Bethe-Salpeter Equation)**: Includes excitonic effects in optical absorption

**Prerequisites:**
- CrystalMath with YAMBO support (`pip install crystalmath[all]`)
- Access to beefcake2 cluster with YAMBO installed

## 1. Setup and Imports

In [ ]:
# Import required modules
from crystalmath.high_level import HighThroughput
from crystalmath.high_level.runners import OpticalAnalysis
from crystalmath.high_level.clusters import get_cluster_profile

from pymatgen.core import Structure
import matplotlib.pyplot as plt
import numpy as np

print("Imports successful!")

In [ ]:
# Load structure for optical calculations
# We'll use a 2D semiconductor (NbOCl2) as an example
# You can also use mp-ids or CIF files

structure_path = "NbOCl2.cif"  # Replace with your structure
structure = Structure.from_file(structure_path)

print(f"Structure: {structure.composition.reduced_formula}")
print(f"Number of atoms: {len(structure)}")
print(f"Space group: {structure.get_space_group_info()[0]}")

## 2. Run GW Calculation

The GW approximation provides accurate quasiparticle energies. We'll run a complete
DFT -> GW workflow using VASP for DFT and YAMBO for GW.

**GW Protocols:**
- `g0w0`: Single-shot GW (fastest, ~0.1-0.2 eV accuracy)
- `gw0`: Partially self-consistent (recommended, better accuracy)
- `evgw`: Eigenvalue self-consistent (most accurate, slowest)

In [ ]:
# Method 1: Using HighThroughput API (simplest)
results = HighThroughput.run_standard_analysis(
    structure=structure_path,
    properties=["bands", "gw"],
    codes={"dft": "vasp", "gw": "yambo"},
    cluster="beefcake2",
    protocol="moderate"
)

print(f"GW calculation completed!")
print(f"DFT band gap: {results.band_gap_ev:.3f} eV")
print(f"GW band gap: {results.gw_gap_ev:.3f} eV")

In [ ]:
# Method 2: Using OpticalAnalysis runner for full control
cluster = get_cluster_profile("beefcake2")

runner = OpticalAnalysis(
    cluster=cluster,
    dft_code="vasp",
    gw_code="yambo",
    gw_protocol="gw0",          # Partially self-consistent
    n_bands_gw=100,              # Number of bands for GW
    include_bse=False,           # Just GW for now
    protocol="moderate",
    output_dir="./NbOCl2_gw"
)

results = runner.run(structure_path)

print(f"\nGW Results:")
print(f"  DFT gap: {results.band_gap_ev:.3f} eV")
print(f"  GW gap: {results.gw_gap_ev:.3f} eV")
print(f"  GW correction: {results.gw_gap_ev - results.band_gap_ev:.3f} eV")

## 3. Run BSE for Optical Absorption

The BSE calculation builds on GW results to include electron-hole interactions (excitons).
This gives accurate optical absorption spectra.

In [ ]:
# Run full GW+BSE workflow
results = HighThroughput.run_standard_analysis(
    structure=structure_path,
    properties=["bands", "gw", "bse"],
    codes={"dft": "vasp", "gw": "yambo"},
    cluster="beefcake2",
    protocol="moderate"
)

print(f"GW+BSE calculation completed!")
print(f"DFT band gap: {results.band_gap_ev:.3f} eV")
print(f"GW band gap: {results.gw_gap_ev:.3f} eV")
print(f"Optical gap: {results.optical_gap_ev:.3f} eV")

In [ ]:
# Using OpticalAnalysis runner with BSE
runner = OpticalAnalysis(
    cluster=cluster,
    dft_code="vasp",
    gw_code="yambo",
    gw_protocol="gw0",
    n_bands_gw=100,
    n_valence_bse=4,             # Valence bands in BSE
    n_conduction_bse=4,          # Conduction bands in BSE
    include_bse=True,
    protocol="moderate",
    output_dir="./NbOCl2_bse"
)

results = runner.run(structure_path)

print(f"\nOptical Properties:")
print(f"  DFT gap: {results.band_gap_ev:.3f} eV")
print(f"  GW gap: {results.gw_gap_ev:.3f} eV")
print(f"  Optical gap (BSE): {results.optical_gap_ev:.3f} eV")
print(f"  Exciton binding: {results.exciton_binding_ev:.3f} eV")

## 4. Plot Optical Spectrum

Visualize the optical absorption spectrum from BSE calculations.

In [ ]:
# Plot optical absorption spectrum
fig = results.plot_optical()
plt.show()

In [ ]:
# Customized optical spectrum plot
fig, ax = plt.subplots(figsize=(10, 6))

results.plot_optical(
    ax=ax,
    component='xx',              # Polarization direction
    energy_range=(0, 6),         # Energy range in eV
    color='#2E86AB',
    linewidth=2
)

# Mark important energies
ax.axvline(x=results.band_gap_ev, color='gray', linestyle='--', 
           label=f'DFT gap: {results.band_gap_ev:.2f} eV')
ax.axvline(x=results.gw_gap_ev, color='green', linestyle='--', 
           label=f'GW gap: {results.gw_gap_ev:.2f} eV')
ax.axvline(x=results.optical_gap_ev, color='red', linestyle='-', 
           label=f'Optical gap: {results.optical_gap_ev:.2f} eV')

ax.set_xlabel('Photon Energy (eV)', fontsize=12)
ax.set_ylabel('Absorption (a.u.)', fontsize=12)
ax.set_title(f'Optical Absorption: {results.formula}', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('optical_absorption.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot multiple polarization directions for anisotropic materials
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

components = ['xx', 'yy', 'zz']
colors = ['#E74C3C', '#2ECC71', '#3498DB']

for ax, comp, color in zip(axes, components, colors):
    results.plot_optical(
        ax=ax,
        component=comp,
        energy_range=(0, 5),
        color=color,
        linewidth=1.5
    )
    ax.set_title(f'Polarization: {comp}')
    ax.set_xlabel('Energy (eV)')

axes[0].set_ylabel('Absorption (a.u.)')

plt.suptitle(f'Anisotropic Optical Absorption: {results.formula}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Compare DFT vs GW Gaps

Let's visualize the band gap corrections from GW.

In [ ]:
# Bar chart comparing gaps
fig, ax = plt.subplots(figsize=(8, 6))

gaps = {
    'DFT (PBE)': results.band_gap_ev,
    'GW': results.gw_gap_ev,
    'BSE (Optical)': results.optical_gap_ev
}

colors = ['#95A5A6', '#27AE60', '#E74C3C']

bars = ax.bar(gaps.keys(), gaps.values(), color=colors, edgecolor='black', linewidth=1.2)

# Add value labels on bars
for bar, value in zip(bars, gaps.values()):
    height = bar.get_height()
    ax.annotate(f'{value:.2f} eV',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Band Gap (eV)', fontsize=12)
ax.set_title(f'Band Gap Comparison: {results.formula}', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(gaps.values()) * 1.2)

plt.tight_layout()
plt.savefig('gap_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Detailed gap analysis
print(f"{'='*60}")
print(f"Band Gap Analysis for {results.formula}")
print(f"{'='*60}")

print(f"\nEnergy Gaps:")
print(f"  DFT (PBE) gap:      {results.band_gap_ev:.3f} eV")
print(f"  GW gap:             {results.gw_gap_ev:.3f} eV")
print(f"  Optical gap (BSE):  {results.optical_gap_ev:.3f} eV")

print(f"\nCorrections:")
gw_correction = results.gw_gap_ev - results.band_gap_ev
print(f"  GW correction (GW - DFT):     {gw_correction:+.3f} eV")
print(f"  Exciton binding energy:       {results.exciton_binding_ev:.3f} eV")

print(f"\nPhysical Interpretation:")
print(f"  - DFT typically underestimates gaps (missing self-energy effects)")
print(f"  - GW adds quasiparticle corrections (~{gw_correction/results.band_gap_ev*100:.0f}% increase here)")
print(f"  - BSE optical gap < GW gap due to exciton binding")
print(f"  - Exciton binding = GW gap - Optical gap = {results.exciton_binding_ev:.3f} eV")

## 6. Extract Exciton Binding Energy

The exciton binding energy is the energy required to dissociate an electron-hole pair.

In [ ]:
# Calculate exciton binding energy
# Eb = E_gap(GW) - E_gap(optical)

exciton_binding = results.gw_gap_ev - results.optical_gap_ev

print(f"Exciton Binding Energy Analysis")
print(f"{'='*40}")
print(f"  GW gap:             {results.gw_gap_ev:.3f} eV")
print(f"  Optical gap:        {results.optical_gap_ev:.3f} eV")
print(f"  Exciton binding:    {exciton_binding:.3f} eV")

# Interpret the binding energy
if exciton_binding > 0.5:
    print(f"\n  -> Strong exciton binding (likely 2D or quantum-confined system)")
elif exciton_binding > 0.1:
    print(f"\n  -> Moderate exciton binding (Frenkel or intermediate excitons)")
else:
    print(f"\n  -> Weak exciton binding (Wannier-Mott excitons, typical for bulk)")

In [ ]:
# Visualize the energy levels
fig, ax = plt.subplots(figsize=(8, 6))

# Energy levels
y_positions = [0, results.optical_gap_ev, results.gw_gap_ev]
labels = ['VBM', 'Exciton', 'CBM (GW)']
colors = ['#3498DB', '#E74C3C', '#2ECC71']

# Draw energy levels
for y, label, color in zip(y_positions, labels, colors):
    ax.axhline(y=y, xmin=0.2, xmax=0.8, color=color, linewidth=3)
    ax.text(0.85, y, f'{label}\n{y:.2f} eV', va='center', fontsize=11)

# Draw exciton binding energy arrow
ax.annotate('', xy=(0.5, results.optical_gap_ev), xytext=(0.5, results.gw_gap_ev),
            arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
ax.text(0.52, (results.optical_gap_ev + results.gw_gap_ev)/2, 
        f'Eb = {exciton_binding:.2f} eV', fontsize=11, color='purple')

ax.set_xlim(0, 1)
ax.set_ylim(-0.5, results.gw_gap_ev + 0.5)
ax.set_ylabel('Energy (eV)', fontsize=12)
ax.set_title(f'Energy Level Diagram: {results.formula}', fontsize=14, fontweight='bold')
ax.set_xticks([])

plt.tight_layout()
plt.savefig('exciton_binding.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Export Results

In [ ]:
# Export to various formats
# JSON
results.to_json('optical_results.json')
print("Results saved to optical_results.json")

# CSV
df = results.to_dataframe()
df.to_csv('optical_results.csv', index=False)
print("Results saved to optical_results.csv")

# LaTeX table
latex = results.to_latex_table()
with open('optical_table.tex', 'w') as f:
    f.write(latex)
print("LaTeX table saved to optical_table.tex")

In [ ]:
# View LaTeX table
print("\nLaTeX Table:")
print(latex)

## Summary

In this notebook, you learned how to:

1. **Run GW calculations** for accurate quasiparticle band gaps
2. **Calculate BSE optical spectra** with excitonic effects
3. **Plot optical absorption** with customization options
4. **Compare DFT vs GW** band gaps quantitatively
5. **Extract exciton binding energy** from GW and BSE results

**Key Concepts:**
- DFT systematically underestimates band gaps
- GW provides quasiparticle corrections (~20-100% increase)
- BSE includes electron-hole interactions (excitons)
- Exciton binding = GW gap - optical gap

**Computational Notes:**
- GW/BSE calculations are computationally expensive
- GPU acceleration (YAMBO with CUDA) significantly speeds up calculations
- Convergence testing is important for reliable results

**Next:** See `05_high_throughput.ipynb` for batch analysis of multiple materials.